In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from datetime import datetime
import numpy as np
from googlesearch import search
import re

In [2]:
def fghtdta(name):
    
    #name="tony ferguson"
    query1 = "wikipedia"+ name
    query2 = "ufcstats com"+ name
    query3 = "fightmatrix"+ name
    
    urls1=[]
    urls2=[] 
    urls3=[] 
    
    for j in search(query1, tld="co.in", num=10, stop=1, pause=2): 
        urls1.append(j)
    for j in search(query2, tld="co.in", num=10, stop=1, pause=2): 
        urls2.append(j)
    for j in search(query3, tld="co.in", num=10, stop=1, pause=2): 
        urls3.append(j)
        
    url1 = urls1[0]
    url2 = urls2[0]
    url3 = urls3[0]
    
    req1 = urllib.request.urlopen(url1)
    article1 = req1.read().decode()
    soup1 = BeautifulSoup(article1, 'html.parser')
    
    req2 = urllib.request.urlopen(url2)
    article2 = req2.read().decode()
    soup2 = BeautifulSoup(article2, 'html.parser')
    
    
    req3 = urllib.request.urlopen(url3)
    article3 = req3.read().decode()
    soup3 = BeautifulSoup(article3, 'html.parser')
    tables = soup1.find_all('table', class_='wikitable')
    tr=tables[1].find_all('tr')
    X1= []
    X2= []
    X3= []
    X4= []
    X5= []
    X6= []
    X7= []
    for items in tr[1:]:
        info=items.find_all('td')
        X1.append(info[0].text.strip())
        X2.append(info[1].text.strip())
        X3.append(info[2].text.strip())
        X4.append(info[3].text.strip())
        X5.append(info[4].text.strip())
        X6.append(info[5].text.strip())
        X7.append(info[6].text.strip())
    tab1= pd.DataFrame (list(zip(X1,X2,X3,X4,X5,X6,X7)))
    index=[]
    for i in range(7):
        index.append(tr[0].find_all('th')[i].text.strip())
    tab1.columns=index
    ageatfight=[]
    for date in tab1["Date"]:
        ageatfight.append((datetime.strptime(date, '%B %d, %Y')-datetime.strptime(soup1.find_all('span', class_="bday")[0].text.strip(), '%Y-%m-%d')).days/365)
    tab1["ageatfight"]=ageatfight
    timeout=[0]
    for i in np.flip(range(len(tab1)))[:-1]:
            timeout.append((datetime.strptime(tab1["Date"][i-1], '%B %d, %Y')-datetime.strptime(tab1["Date"][i], '%B %d, %Y')).days/365)
    tab1["timeout"]=np.flip(timeout)
    tab1["Round"]= list(map(int, tab1["Round"])) 
    cumrnds=np.cumsum(tab1["Round"],axis=0) 
    cumrnds=np.flip(cumrnds, axis=0).values
    tab1["cumrnds"]=cumrnds
    stats=soup2.find_all('li')
    X8 = []
    X9 = []
    for items in stats[3:16]:
        X8.append(items.text.strip()[-8:])
        X9.append(items.text.strip()[:5])
    for i in range (len(X8)):
        tab1[X9[i]]=np.repeat(X8[i],len(tab1))
    stats2=soup3.find_all('td', class_="tdRank")[1].find_all('div')
    X10 = []
    X11 = []
    for items in stats2:
        X11.append(items.text.strip().split(':')[0])
        X10.append(items.text.strip().split(':')[1])
    for i in range (len(X10)):
        tab1[X11[i]]=np.repeat(X10[i],len(tab1))
    X12 = []
    X13 = []
    for rec in tab1['Record']:
        X12.append(rec.split('–')[0])
        X13.append(rec.split('–')[1])
    X14 = []
    X15 = []
    for rec in tab1["'Big League' Record"]:
        X14.append(rec.split('-')[0])
        X15.append(rec.split('-')[1])
    X16 = []
    X17 = []
    for rec in tab1["Last 3 Years"]:
        X16.append(rec.split('-')[0])
        X17.append(rec.split('-')[1])
    X18 = []
    for i in range(len(tab1['Last Quality Perf.'])):
        X18.append((datetime.strptime(tab1['Date'][i], '%B %d, %Y')-datetime.strptime(tab1['Last Quality Perf.'][i][1:], '%Y-%m-%d')).days>180)
    tab1['W1']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X12]
    tab1['W2']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X13]
    tab1['W3']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X14]
    tab1['L1']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X15]
    tab1['L2']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X16]
    tab1['L3']=[re.sub("[\(\[].*?[\)\]]", "", value) for value in X17]
    tab1['pstprm']=[value*1 for value in X18]
    alphanumeric = [character for character in tab1['Heigh'][0] if character.isalnum()]
    tab1['Heigh']=int(alphanumeric[0])*30.48+int("".join(alphanumeric[1:]))*2.54
    tab1['Win Finish %']=[float(value[:-1]) for value in tab1['Win Finish %']]
    tab1['Str. ']=[float(value[:-1]) for value in tab1['Str. ']]
    tab1['TD Ac']=[float(value[:-1]) for value in tab1['TD Ac']]
    tab1['TD De']=[float(value[:-1]) for value in tab1['TD De']]
    tab1['Quality Perf. %']=[float(value[:-1]) for value in tab1['Quality Perf. %']]
    tab1['Reach']=[float(value[:-1])*2.54 for value in tab1['Reach']]
    tab1['Ape']=tab1['Reach']/tab1['Heigh']
    tab1['Weigh']=[float(value[:4]) for value in tab1['Weigh']]
    tab1['Record'] = tab1['Record'].shift(-1)
    tab1['ageatfight'] = tab1['ageatfight'].shift(-1)
    tab1['cumrnds'] = tab1['cumrnds'].shift(-1)
    tab1['W1']=tab1['W1'].shift(-1)
    tab1['W2']=tab1['W2'].shift(-1)
    tab1['W3']=tab1['W3'].shift(-1)
    tab1['L1']=tab1['L1'].shift(-1)
    tab1['L2']=tab1['L2'].shift(-1)
    tab1['L3']=tab1['L3'].shift(-1)
    return tab1

In [51]:
def nestedfightdta(name):
    data=fghtdta(name)
    Opponents=data["Opponent"][0:len(data["Opponent"])]
    X=[]
    for op in Opponents:
        try:
            X.append(fghtdta(op))
        except:
            pass
    Y=pd.concat([x[x["Opponent"].apply(lambda x: x.lower())==name.lower()] for x in X], axis=0)
    Z=data.merge(Y, left_on='Date', right_on='Date',suffixes=('_fighter', '_opponent'))
    return Z

In [52]:
nestedfightdta

<function __main__.nestedfightdta(name)>

In [53]:
X=nestedfightdta("tony ferguson")

In [54]:
X

,Res._fighter,Record_fighter,Opponent_fighter,Method_fighter,Event_fighter,Date,Round_fighter,ageatfight_fighter,timeout_fighter,cumrnds_fighter,...,540 Metric_opponent,Rating Points_opponent,W1_opponent,W2_opponent,W3_opponent,L1_opponent,L2_opponent,L3_opponent,pstprm_opponent,Ape_opponent
0,Win,23–3,Anthony Pettis,TKO (corner stoppage),UFC 229,"October 6, 2018",2,33.673973,0.997260,54.0,...,.667,415,21,7,14,10,3,4,0,1.028571
1,Win,21–3,Rafael dos Anjos,Decision (unanimous),The Ultimate Fighter Latin America 3 Finale: d...,"November 5, 2016",5,32.438356,0.315068,51.0,...,.733,635,25,8,20,11,4,4,0,1.029412
2,Win,20–3,Lando Vannata,Submission (D'Arce choke),UFC Fight Night: McDonald vs. Lineker,"July 13, 2016",2,31.849315,0.589041,48.0,...,.550,206,8,0,4,4,2,2,0,1.028986
3,Win,17–3,Gleison Tibau,Submission (rear-naked choke),UFC 184,"February 28, 2015",1,30.835616,0.230137,43.0,...,.650,230,33,10,16,12,2,2,0,1.014286
4,Win,10–2,Ramsey Nijem,KO (punch),The Ultimate Fighter: Team Lesnar vs. Team dos...,"June 4, 2011",1,26.635616,0.690411,27.0,...,.684,79,4,1,5,5,1,2,0,1.056338
